## As requested, run a simulation notebook to reduce calculation needs

In [7]:
# plotly import
import pandas as pd
import numpy as np
import scipy as sp
import plotly
# Plotly's Python API contains a figure factory module which includes many wrapper functions 
# that create unique chart types that are not yet included in plotly.js. 
import plotly.figure_factory as ff
# REF: https://plot.ly/python/reference/
import plotly.graph_objs as go
# offline plotting
plotly.offline.init_notebook_mode()
import plotly.offline as py

# customised block
import numpy as np
from CNC import chiral_nematic_simulation_method
import matplotlib.pyplot as plt


ImportError: C extension: 'hashtable' not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext --inplace --force' to build the C extensions first.

### CNC_water_infiltration_simulation_method class

In [2]:
class CNC_water_infiltration_simulation_method():
    def __init__(self):
        self.water = 1.33
        # input value to change this
        self.CNC = (1.51, 1.59) 
        self.original_pitch = 300e-9
        self.original_thickness = 3500e-9
        self.water_infiltration_fractions = [0,0.06,0.14,0.26,0.35,0.44]
    
    def CNC_water_mixture_refractive_index(self, water_volume_fraction):
        'Calculate the new refractive index based on volume fraction'
        # dielectric constant epsilon is sqrt of refractive index
        # for composite materials, dielectric constant calculation: 
        # e = e1*v1 + e2*v2  v is the volume fraction   http://www.sciencedirect.com/science/article/pii/S0030401899006951
        n_CNC_water_mixture  = (np.sqrt(np.square(self.CNC[0])*(1-water_volume_fraction)+np.square(self.water)*water_volume_fraction),
                            np.sqrt(np.square(self.CNC[1])*(1-water_volume_fraction)+np.square(self.water)*water_volume_fraction))
        print(n_CNC_water_mixture)
        return n_CNC_water_mixture

    def pitch_change_only(self):
        ' for swelling with only pitch change (the number of repeats stayed the same)'
        simulation_parameters_set = {}
        for water_volume_fraction in self.water_infiltration_fractions:
            swollen_thickness = self.original_thickness/(1-water_volume_fraction)
            swollen_pitch = swollen_thickness/(self.original_thickness/self.original_pitch)
            # unchanged refractive index
            n_swollen_CNC_water_mixture = self.CNC
            # create a dictionary to store n, pitch and thickness for corresponding water_volume_fraction
            simulation_parameters_set[water_volume_fraction] = {'n': n_swollen_CNC_water_mixture, 'pitch': swollen_pitch, 'thickness': swollen_thickness, 'rotation': 0}
        return simulation_parameters_set

    def refractive_index_change_only(self):
        ' for swelling with only refractive index change (the thickness unchanged)'
        simulation_parameters_set = {}
        for water_volume_fraction in self.water_infiltration_fractions:
            # unchanged thickness and pitch (filling the pores)
            swollen_thickness = self.original_thickness
            swollen_pitch = self.original_pitch
            n_swollen_CNC_water_mixture = self.CNC_water_mixture_refractive_index(water_volume_fraction)
            # create a dictionary to store n, pitch and thickness for corresponding water_volume_fraction
            simulation_parameters_set[water_volume_fraction] = {'n': n_swollen_CNC_water_mixture, 'pitch': swollen_pitch, 'thickness': swollen_thickness, 'rotation': 0}
        return simulation_parameters_set

    def pitch_change_refractive_index_change(self):
        'for swelling with both refractive index change and pitch change'
        simulation_parameters_set = {}
        for water_volume_fraction in self.water_infiltration_fractions:
            swollen_thickness = self.original_thickness/(1-water_volume_fraction)
            swollen_pitch = swollen_thickness/(self.original_thickness/self.original_pitch)
            n_swollen_CNC_water_mixture = self.CNC_water_mixture_refractive_index(water_volume_fraction)
            # create a dictionary to store n, pitch and thickness for corresponding water_volume_fraction
            simulation_parameters_set[water_volume_fraction] = {'n': n_swollen_CNC_water_mixture, 'pitch': swollen_pitch, 'thickness': swollen_thickness, 'rotation': 0}
        return simulation_parameters_set
        

## Actual water infiltration simulation

In [ ]:
    # Simulation of CNC domain or stacks of domains with z-twist
    # fun!
    chiral_nematic_simulation = chiral_nematic_simulation_method()
    # Normally use it 0~1, higher the number, slower the simulation
    chiral_nematic_simulation.simulation_accuracy = 0.5
    # range of wavelength to be simulated
    chiral_nematic_simulation.set_wave_vector(wavelength_range=(400e-9, 900e-9), n_wavelengths=200)
    # simulation conditions: refer to chiral_nematic_simulation.calculate_structure()
    chiral_nematic_simulation.simulation_modes = ['R_LL']


    # get parameters form water swelling
    CNC_water_infiltration_simulation = CNC_water_infiltration_simulation_method()
    # define some initial parameters
    CNC_water_infiltration_simulation.CNC = (1.515, 1.585)
    CNC_water_infiltration_simulation.original_pitch = 294e-9
    CNC_water_infiltration_simulation.original_thickness = 4000e-9
    CNC_water_infiltration_simulation.water_infiltration_fractions = [0,0.087, 0.178, 0.262, 0.323, 0.375, 0.43]

    # type of water swelling mechanisms 
    #simulation_parameters_set = CNC_water_infiltration_simulation.pitch_change_refractive_index_change()
    simulation_parameters_set = CNC_water_infiltration_simulation.pitch_change_refractive_index_change()


    # the folder to save csv
    chiral_nematic_simulation.output_filename = 'pitch_change_refractive_index_change'
    chiral_nematic_simulation.folder = r'C:\Users\herbz\Documents\GitHub\PhD-python\model the light\Berreman4x4-master\CNC simulation\csv'
    
    # simulation
    layer1_parameters_set = simulation_parameters_set
    layer2_parameters_set = None
    interface_parameters_set = None
    chiral_nematic_simulation.multiple_materials_layers_parameter_sets(
                                        layer1_parameters_set=layer1_parameters_set,
                                        layer2_parameters_set=layer2_parameters_set,
                                        interface_parameters_set=interface_parameters_set)

### Plotting simulated result
### **to plot, run the last cell first**

In [2]:
# traces is single set of data plotted as Scatter, Bar, ... 
simulation_data = []
for condition in chiral_nematic_simulation.all_data:
    for mode in chiral_nematic_simulation.simulation_modes:
        # convert wavelength to nm
        wavelength_list_nm = chiral_nematic_simulation.wavelength_list * 1e9
        trace = go.Scatter(
            x=wavelength_list_nm,
            # B4x4 method, get(R_RR)
            y=chiral_nematic_simulation.all_data[condition].get(mode),
            name = '{}_{}'.format(condition,mode),
            mode = 'lines',
        )
        # data is list of all the traces
        simulation_data.append(trace)

data = simulation_data
# specify layout REF:  https://plot.ly/python/reference/#layout
# layout comes from previous block

# Also see subplots below
fig = go.Figure(data=data, layout=layout)

# plot offline
#py.iplot(fig, filename='no subplot')

NameError: name 'chiral_nematic_simulation' is not defined

### Plot CSV data and compare

In [3]:
# read data from csv via pandas
humidity_df = pd.read_csv("humidity_spec.csv")
table = ff.create_table(humidity_df[:5])
csv_data = [table]
py.iplot(table, filename='jupyter/table1')

# traces is single set of data plotted as Scatter, Bar, ... 
humidity_data = []
# list of all the columns, allowing us to select 1,3,5 for X and 2,4,6 for Y
column_list = list(humidity_df)

for i in range(int(len(column_list)/2)):
    i+=1
    normalised_y = humidity_df[column_list[2*i-1]] - 0.05
    trace = go.Scatter(
        x=humidity_df[column_list[2*i-2]],
        y=normalised_y,
        name = '{}'.format(column_list[2*i-1]),
        mode = 'lines'
        )
    # data is list of all the traces
    humidity_data.append(trace)
    
data = simulation_data + humidity_data
# specify layout REF:  https://plot.ly/python/reference/#layout
# layout comes from previous block

# Also see subplots below
fig = go.Figure(data=data, layout=layout)

# plot offline
py.iplot(fig, filename='no subplot')

FileNotFoundError: File b'humidity_spec.csv' does not exist

## Plot.ly plotting
### Layout settings for plotting

In [8]:
layout = dict(
                title = 'Styled Scatter',
                # global font control
                font=dict(
                    family='Arial, sans-serif',
                    size=20,
                    color='black'
                ),

                xaxis = dict(
                    title = 'x_title / nm',
                    # range
                    #range=[0, 10],
                    autorange=True,
                    # ticks
                    #tickmode='linear',
                    #dtick=5,
                    tickmode='auto',
                    ticks="inside",
                    tickwidth=2,
                    tickfont=dict(
                    family='Arial, sans-serif',
                    size=14,
                    color='black'
                    ),
                    # mirror the axis and ticks on the top
                    mirror="ticks",
                    zeroline = False,
                    showline=True,
                    showgrid=False, 
                    linewidth=2,
                    zerolinewidth=5,
                    ),
    
                yaxis = dict(
                    title = 'y_label / nm',
                    # range
                    range=[0, 20],
                    autorange=True,
                    # ticks
                    #tickmode='linear',
                    #dtick=5,
                    tickmode='auto',
                    ticks="inside",
                    tickwidth=2,
                    tickfont=dict(
                    family='Arial, sans-serif',
                    size=14,
                    color='black'
                    ),
                    # mirror the axis and ticks on the top
                    mirror="ticks",
                    zeroline = False,
                    showline=True,
                    showgrid=False, 
                    linewidth=2,
                    zerolinewidth=5,
                    ),
    
                # legend
                showlegend=True,
                legend = dict(
                    x=3,
                    y=1,
                    traceorder='normal',
                    font=dict(
                        family='Arial, sans-serif',
                        size=20,
                        #color='black'
                    ),
                    #
                    bgcolor='rgba(0,0,0,0)',
                    #bordercolor='black',
                    #borderwidth=0.5
                    ),
    
                # figure size
                height=600, 
                width=600,
                margin=go.Margin(
                    l=60,
                    r=50,
                    b=50,
                    t=50,
                    pad=0,
                    autoexpand=False,
                    ),
                )
